general idea: image is 32x32x4. certain features are also 32x32x4.

why not concatenate all features into the image so it becomes 32x32x16, and then run unet on that?

In [1]:
import numpy as np # linear algebra
import scipy as scipy
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import dill

import keras as k
import cv2
from tqdm import tqdm


Using Theano backend.


In [2]:
with open('tiffs.pkl', 'rb') as in_strm:
    all_info = dill.load(in_strm)
x_train = np.array(all_info[0], dtype=np.float64)
x_test = np.array(all_info[1], dtype=np.float64)
y_train = all_info[2]
y_test = all_info[3]
label_map = all_info[4]
inv_label_map = all_info[5]
print(x_train.shape)

(35000, 32, 32, 4)


In [3]:
with open('basicFeatures.pkl', 'rb') as in_strm:
    features = dill.load(in_strm)

In [4]:
train_features = features[:8]
test_features = features[8:]

for i, train_feature in enumerate(train_features):
    train_feature = np.array(train_feature, dtype=np.float64)
    test_feature = np.array(test_features[i], dtype=np.float64)
    mean = np.mean(train_feature, axis = 0)
    train_feature -= mean # zero-center
    test_feature -= mean
    std = np.std(train_feature, axis = 0)
    train_feature /= std # normalize
    test_feature /= std
    train_features[i] = np.nan_to_num(train_feature)
    test_features[i] = np.nan_to_num(test_feature)
    
mean = np.mean(x_train, axis = 0)
x_train -= mean # zero-center
x_test -= mean
std = np.std(x_train, axis = 0)
x_train /= std # normalize
x_test /= std

#put into network
train_input = [x_train]
for train_feature in train_features:
    train_input.append(train_feature)

test_input = [x_test]
for test_feature in test_features:
    test_input.append(test_feature)

In [10]:
x_train_with_features = []
for i in range(x_train.shape[0]):
    x_train_with_features.append(np.concatenate((x_train[i], train_features[5][i],
                                                 train_features[6][i], train_features[7][i]), axis=2))

In [11]:
with open('x_train_with_features.pkl', 'wb') as out_strm: 
    dill.dump(x_train_with_features, out_strm)

In [12]:
x_train_with_features = []
train_features = []
features = []

In [13]:
x_test_with_features = []
for i in range(x_test.shape[0]):
    x_test_with_features.append(np.concatenate((x_test[i], test_features[5][i],
                                                 test_features[6][i], test_features[7][i]), axis=2))

In [14]:
with open('x_test_with_features.pkl', 'wb') as out_strm: 
    dill.dump(x_test_with_features, out_strm)